# Gemma3 through vLLM on Sagemaker

## Gemma3 : SLM(Small Language Model) 
[특징]
- 다양한 모델 크기 : 1B, 4B, 12B, 27B 
- 다국어 지원 : 140개 이상의 언어 지원, 특히 한국어 성능 우수
- 멀티모달 기능 : 텍스트뿐만 아니라 이미지, 비디오 분석 가능
- 확장된 컨텍스트 윈도우 : 최대 128K 토큰 처리 가능 
- 경량화 및 효율성 : 단일 GPU/TPU 및 저사양 기기에서도 높은 성능 발휘


## Use DJL with the SageMaker Python SDK
- SageMaker Python SDK를 사용하면 Deep Java Library를 이용하여 Amazon SageMaker에서 모델을 호스팅할 수 있습니다.
- Deep Java Library (DJL) Serving은 DJL이 제공하는 고성능 범용 독립형 모델 서빙 솔루션입니다. DJL Serving은 다양한 프레임워크로 학습된 모델을 로드하는 것을 지원합니다.
- SageMaker Python SDK를 사용하면 DeepSpeed와 HuggingFace Accelerate와 같은 백엔드를 활용하여 DJL Serving으로 대규모 모델을 호스팅할 수 있습니다.
- DJL Serving의 지원 버전에 대한 정보는 AWS 문서를 참조하십시오.
- 최신 지원 버전을 사용하는 것을 권장합니다. 왜냐하면 그곳에 우리의 개발 노력이 집중되어 있기 때문입니다.
- SageMaker Python SDK 사용에 대한 일반적인 정보는 SageMaker Python SDK 사용하기를 참조하십시오.
> REF: [BLOG] Deploy LLM with vLLM on SageMaker in only 13 lines of code


> DJLServing LMI 이미지 리스트 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md

> HF 모델에 따라, Huggingface에서 사용 License를 요구하는 모델이 있습니다. Gemma3의 경우 라이선스를 요구 합니다. huggingface 참조

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

%pip install -qU sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
autogluon-multimodal 1.2 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.55.1 which is incompatible.
autogluon-timeseries 1.2 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.55.1 which is incompatible.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
spacy 3.8.5 requires thinc<8.4.0,>=8.3.4, but you have thinc 8.3.

### 1. Depoly model on SageMaker

In [4]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [5]:
role = get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

### Setup Configuration

최신 버전 (DJL Serving 0.33)을 사용하는 경우, `sagemaker.image_uris.retrieve()`가 늦게 반영될 수도 있다. 

이 경우, container_uri 자체를 사용 하도록 합니다.  

ex) https://github.com/aws/deep-learning-containers/blob/master/available_images.md

> * 참조 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [6]:
base_model = "Gemma-3"
model_id = "google/gemma-3-12b-pt"
instance_type = "ml.g5.24xlarge"

# 최신번전 : version 0.33 할려면, URI를 직접 사용하거나,  --> sagemaker upgrade 필요 : `uv pip install -U sagemaker`
# container_uri = sagemaker.image_uris.retrieve(
#     framework="djl-lmi", version="0.30.0", region=region
# )
# container_uri
container_uri = "763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128"
# 참조 : https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [7]:
container_startup_health_check_timeout = 900

endpoint_name = sagemaker.utils.name_from_base(base_model)

print (f'container_uri: {container_uri}')
print (f'container_startup_health_check_timeout: {container_startup_health_check_timeout}')
print (f'instance_type: {instance_type}')
print (f'endpoint_name: {endpoint_name}')

container_uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.33.0-lmi15.0.0-cu128
container_startup_health_check_timeout: 900
instance_type: ml.g5.24xlarge
endpoint_name: Gemma-3-2025-08-13-02-12-55-046


### Creat model with env variables

- Target model: DeepSeek-Coder-V2-Light-Instruct
- Backend for attention computation in vLLM
- Available options:
    - "TORCH_SDPA": use torch.nn.MultiheadAttention
    - "FLASH_ATTN": use FlashAttention
    - "XFORMERS": use XFormers
    - "ROCM_FLASH": use ROCmFlashAttention
    - "FLASHINFER": use flashinfer


- '"OPTION_DISABLE_FLASH_ATTN": "false"' is for HF Accelerate with Seq-Scheduler
- It will be ignored when using vLLM beckend

> [DOC] DJL-Container and Model Configurations (info. about properties)

> [DOC] Backend Specific Configurations

In [8]:
HF_TOKEN = os.environ.get("HF_TOKEN","")
deploy_env = {
    "HF_MODEL_ID": model_id,
    "OPTION_ROLLING_BATCH": "vllm",
    "OPTION_TENSOR_PARALLEL_DEGREE": "max",
    "OPTION_MAX_ROLLING_BATCH_SIZE": "64",
    "OPTION_DTYPE":"fp16",
    "OPTION_TRUST_REMOTE_CODE": "true",
    "OPTION_MAX_MODEL_LEN": "4096",
    # "VLLM_ATTENTION_BACKEND": "XFORMERS", # meta의 경우 XFORMERS, FlashAttention (default)
    #"OPTION_DISABLE_FLASH_ATTN": "false", ## HF Accelerate with Seq-Scheduler
    "HF_TOKEN": HF_TOKEN, # "<your token>"
}

In [9]:
model = sagemaker.Model(
    image_uri=container_uri,
    role=role,
    env=deploy_env
)

### Deploy Model on Sagemaker

In [10]:
model.deploy(
    instance_type=instance_type,
    initial_instance_count=2,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=container_startup_health_check_timeout,
    sagemaker_session=sagemaker_session
)

----------------------!

### Invocation

In [6]:
# endpoint_name = "Gemma-3-2025-08-12-13-47-53-107"
# endpoint_name = "Gemma-3-2025-08-13-02-12-55-046"
endpoint_name = "gemma3-s3-vllm-async-2025-08-14-02-10-03-005"

In [7]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [9]:
# 호출 예시
response = predictor.predict({
    "inputs": "AI Agent에 대해 100단어 내외로 설명해 주세요.",
    # "inputs": "tell me aboiut the AI Agent.",
    "parameters": {
        "max_new_tokens": 512,
        "stop": ["<|endoftext|>"]  # Stop sequences 지정 가능
    }
})

print(response)

{'generated_text': '\n\nAI 에이전트는 특정 목표를 달성하기 위해 환경과 상호 작용하는 프로그램입니다. 이러한 에이전트는 데이터를 분석하고, 결정을 내리고, 작업을 수행하여 목표를 달성합니다. AI 에이전트는 게임, 추천 시스템, 챗봇 등 다양한 응용 분야에서 사용됩니다. 이들은 인간처럼 생각하고 행동할 수 있도록 설계되었으며, 학습과 적응을 통해 성능을 향상시킬 수 있습니다. 예를 들어, 자율 주행차는 주변 환경을 인식하고, 경로를 계획하고, 안전하게 운전하는 AI 에이전트입니다.\n'}


### Streaming Output

In [10]:
import json

def generate_payload(chat):
        
    # JSON 페이로드 생성
    body = {
        "messages": chat,
        "max_tokens": 512,
        "stream": True,
        "ignore_eos": False
    }
    
    # JSON을 문자열로 변환하고 bytes로 인코딩
    return json.dumps(body).encode('utf-8')

In [11]:
chat = [
    {"role": "system", "content": "너는 질의응답 챗봇입니다. 사용자의 질문의 의도를 파악하여 답변합니다. 답변은 한국어로 합니다"},
    {"role": "user", "content": "AWS AIML Specialist 솔루션즈 아키텍트 역할에 대해 설명해줘"},
]

In [12]:
%%time
# Invoke the endpoint
resp = sm_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name, 
    Body=generate_payload(chat),
    ContentType="application/json"
)
print("Generated response:")
print("-" * 40)

buffer = ""
string = "" 
for event in resp['Body']:
    if 'PayloadPart' in event:
        chunk = event['PayloadPart']['Bytes'].decode()
        buffer += chunk
        try:
            # Try to parse the buffer as JSON
            data = json.loads(buffer)
            if 'choices' in data:
                print(data['choices'][0]['delta']['content'], end='', flush=True)
                string += data['choices'][0]['delta']['content'] 
            buffer = ""  # Clear the buffer after successful parsing
        except json.JSONDecodeError:
            # If parsing fails, keep the buffer for the next iteration
            pass

print("\n" + "-" * 40)

Generated response:
----------------------------------------
AWS AIML (Amazon Interactive Multimedia Learning) Specialist 솔루션즈 아키텍트는 AWS에서 **인터랙티브 학습 경험**을 구축하고 운영하기 위한 전문적인 역할을 담당합니다. 단순히 기술적인 구현에만 집중하는 것이 아니라, 학습 목표, 대상, 콘텐츠 특성 등을 고려하여 최적의 AWS 서비스를 조합하고 아키텍처를 설계하며, 관련 서비스를 통합하여 시너지 효과를 창출합니다. 

좀 더 자세히 살펴보면, 다음과 같은 핵심적인 역할과 책임들을 수행합니다.

**1. 요구사항 분석 및 솔루션 설계:**

* **학습 목표 및 대상 분석:** 고객사의 학습 목표, 학습 대상의 특성 (연령, 배경 지식, 기술 수준 등)을 정확히 파악합니다.
* **콘텐츠 분석:** 학습 콘텐츠의 형태 (동영상, 텍스트, 퀴즈, 시뮬레이션 등) 및 특성을 분석하여 적합한 기술 스택을 결정합니다.
* **AWS 서비스 최적 조합:**  AIML, Amazon Kendra, Amazon Lex, Amazon Polly, Amazon Transcribe, Amazon SageMaker, Amazon CloudFront, Amazon S3 등 다양한 AWS 서비스를 효과적으로 조합하여 최적의 학습 솔루션을 설계합니다.
* **아키텍처 설계:** 확장성, 안정성, 보안, 비용 효율성을 고려하여 견고하고 유지보수가 용이한 아키텍처를 설계합니다.
* **기술 문서 작성:** 설계 내용을 상세하게 문서화하여 개발팀 및 고객사와의 이해도를 높입니다.

**2. 기술 구현 및 통합:**

* **AIML 엔진 구축 및 설정:** AWS AIML 엔진을 설정하고 구성하여 고객사의 학습 콘텐츠에 적용합니다.
* **챗봇 및 음성 인터페이스 구축:** Amazon Lex, Amazon Polly 등을 활용하여 챗봇 또는 음성 인터페이스를 구축하고, 학습 콘텐츠

## AutoScaling

In [ ]:
import pprint
import random

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
    MinCapacity=1,
    MaxCapacity=2

In [ ]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
    PolicyName=policy_name,
    ServiceNamespace="sagemaker",
    ResourceId=resource_id,
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 5.0, # Target for avg invocations per minutes
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
        },
        "ScaleInCooldown": 600, # Duration in seconds until scale in
        "ScaleOutCooldown": 60 # Duration in seconds between scale out
    }
)

In [ ]:
response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

In [ ]:
# 다양한 코딩 태스크를 위한 프롬프트 리스트
prompts = [
    "write a quick sort algorithm in python.",
    "Write a Python function to implement a binary search algorithm.",
    "Create a JavaScript function to flatten a nested array.",
    "Implement a simple REST API using Flask in Python.",
    "Write a SQL query to find the top 5 customers by total purchase amount.",
    "Create a React component for a todo list with basic CRUD operations.",
    "Implement a depth-first search algorithm for a graph in C++.",
    "Write a bash script to find and delete files older than 30 days.",
    "Create a Python class to represent a deck of cards with shuffle and deal methods.",
    "Write a regular expression to validate email addresses.",
    "Implement a basic CI/CD pipeline using GitHub Actions."
]

def generate_payload():
    # 랜덤하게 프롬프트 선택
    prompt = random.choice(prompts)
    
    # JSON 페이로드 생성
    body = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 400,
            # "return_full_text": False  # This does not work with Phi3
        },
        "stream": True,
    }
    
    # JSON을 문자열로 변환하고 bytes로 인코딩
    return json.dumps(body).encode('utf-8')

In [ ]:
%%time
import time

request_duration = 250
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")

while time.time() < end_time:
    payload = generate_payload()
    # Invoke the endpoint
    response = sm_runtime_client.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name, 
        # Body=json.dumps(body), 
        Body = payload,
        ContentType="application/json"
    )

In [ ]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

### Clean Up

In [ ]:

# Delete model
sm_client.delete_model(ModelName=model_name)

# Delete endpoint configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)